In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("/content/drive/MyDrive/SIMPrice.csv")

In [4]:
df

,sim_number,price_vnd
0,343189288,450000
1,888899580,3000000
2,928960006,500000
3,902438679,5000000
4,334307889,450000
...,...,...
199995,866161769,450000
199996,708124126,1000000
199997,904755200,1000000
199998,329220204,450000


In [5]:
df.describe()
# 99000 --> 500000 -> 1000000 -> 5000000 -> 1.680.000.000

,sim_number,price_vnd
count,2.000000e+05,2.000000e+05
mean,7.304797e+08,1.395027e+07
std,2.406479e+08,5.912997e+08
min,3.250098e+08,9.900000e+04
25%,3.922605e+08,5.000000e+05
50%,8.334287e+08,1.000000e+06
75%,9.180874e+08,5.000000e+06
max,9.979797e+08,1.680000e+11


In [6]:
df[df["price_vnd"]<10000000]["price_vnd"].value_counts()

450000     45119
1000000    38472
500000     30619
3000000    29199
5000000    21383
399000       138
299000        91
99000         58
250000        47
119000        32
350000        22
199000        18
290000         9
390000         3
280000         3
400000         2
220000         1
Name: price_vnd, dtype: int64

In [7]:
df[df["price_vnd"]>10000000]["price_vnd"].value_counts()

12000000     5124
11325000     2682
11000000     1760
13000000     1720
15000000     1417
             ... 
130500000       1
12150000        1
80100000        1
424000000       1
439000000       1
Name: price_vnd, Length: 921, dtype: int64

In [8]:
def get_sim_price_category(x):
  if x["price_vnd"]<=450000:
    return 0
  elif x["price_vnd"]==500000:
    return 1
  elif x["price_vnd"]==1000000:
    return 2
  elif x["price_vnd"]==3000000:
    return 3
  elif x["price_vnd"]==5000000:
    return 4
  else:
    return 5

df["sim_price_cat"] = df.apply(get_sim_price_category, axis=1)

In [9]:
df

,sim_number,price_vnd,sim_price_cat
0,343189288,450000,0
1,888899580,3000000,3
2,928960006,500000,1
3,902438679,5000000,4
4,334307889,450000,0
...,...,...,...
199995,866161769,450000,0
199996,708124126,1000000,2
199997,904755200,1000000,2
199998,329220204,450000,0


In [10]:
df["sim_price_cat"].value_counts()

0    45543
2    38472
5    34784
1    30619
3    29199
4    21383
Name: sim_price_cat, dtype: int64

In [11]:
X = []
y = []

for index, row in df.iterrows():
  X.append([int(c) for c in str(row['sim_number'])])
  y.append(row["sim_price_cat"])

X, y = np.array(X), np.array(y)

In [12]:
X.shape

(200000, 9)

In [13]:
y.shape

(200000,)

In [14]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y) # <--- truyền y vào đây
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y_one_hot = onehot_encoder.fit_transform(integer_encoded)
print(y_one_hot) #<--- đầu ra là y_one_hot

[[1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 ...
 [0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]


In [15]:
# Split dữ liệu train, test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.1, random_state=42)

In [16]:
X_train.shape

(180000, 9)

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam


In [18]:
model = Sequential()
# 3 lớp LSTM chồng nhau có trả về sequence
model.add ( LSTM(units=512, return_sequences=True, input_shape=(X_train.shape[1],1)))
model.add (Dropout(0.2))
model.add ( LSTM(units=512, return_sequences=True))
model.add (Dropout(0.2))
model.add ( LSTM(units=512, return_sequences=True))
model.add (Dropout(0.2))
# 1 lớp LSTM không trả về sequence
model.add ( LSTM(units=128, return_sequences=False))
model.add (Dropout(0.2))
# Đưa qua 1 lớp Dense
model.add ( Dense(units=512))
# Lớp output ra kết quả
model.add ( Dense(units=6, activation = "softmax"))

optimizer = Adam()
model.compile(loss = "categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 9, 512)            1052672   
                                                                 
 dropout (Dropout)           (None, 9, 512)            0         
                                                                 
 lstm_1 (LSTM)               (None, 9, 512)            2099200   
                                                                 
 dropout_1 (Dropout)         (None, 9, 512)            0         
                                                                 
 lstm_2 (LSTM)               (None, 9, 512)            2099200   
                                                                 
 dropout_2 (Dropout)         (None, 9, 512)            0         
                                                                 
 lstm_3 (LSTM)               (None, 128)               3

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
model_checkpoint_callback = ModelCheckpoint(
    filepath="ckpt_best.hdf5",
    save_weights_only=True,
    monitor='val_accuracy',
    save_best_only=True, verbose=1)

history = model.fit(X_train, y_train, epochs = 30, batch_size = 64, validation_data = (X_test, y_test), callbacks = [model_checkpoint_callback])
# Val Accuracy : 0.78

In [21]:
model.load_weights("/content/drive/MyDrive/ckpt_best.hdf5")

In [25]:
import random

for i in range(0,5):
  idx = random.randint(0,20000)
  result = model.predict(np.expand_dims(X_test[idx], axis=0))
  print("Số sim = ", X_test[idx])
  print("Kết quả dự đoán = ", result)
  print("Index dự đoán = ", np.argmax(result))
  print("Giá trị thật = ", y_test[idx])
  print("Đúng/sai = ", np.argmax(result) == np.argmax(y_test[idx]))

1/1 [==============================] - 0s 29ms/step
Số sim =  [9 0 6 9 4 3 8 9 1]
Kết quả dự đoán =  [[1.5618983e-03 9.7412431e-01 1.2803834e-02 8.8679716e-03 2.5574476e-03
  8.4634026e-05]]
Index dự đoán =  1
Giá trị thật =  [0. 1. 0. 0. 0. 0.]
Đúng/sai =  True
1/1 [==============================] - 0s 30ms/step
Số sim =  [9 7 5 5 5 2 0 0 6]
Kết quả dự đoán =  [[8.4139731e-07 5.7935281e-08 1.8642049e-07 4.7399382e-05 1.7016554e-03
  9.9824983e-01]]
Index dự đoán =  5
Giá trị thật =  [0. 0. 0. 0. 0. 1.]
Đúng/sai =  True
1/1 [==============================] - 0s 29ms/step
Số sim =  [3 2 9 0 9 0 6 9 3]
Kết quả dự đoán =  [[9.7991014e-01 5.7679382e-03 1.3631926e-02 4.8235344e-04 1.6079682e-04
  4.6821417e-05]]
Index dự đoán =  0
Giá trị thật =  [1. 0. 0. 0. 0. 0.]
Đúng/sai =  True
1/1 [==============================] - 0s 32ms/step
Số sim =  [9 4 1 0 2 6 1 7 3]
Kết quả dự đoán =  [[4.8976723e-08 5.5253046e-04 9.9208915e-01 5.8442228e-03 4.9128459e-04
  1.0228002e-03]]
Index dự đoán =  2
G